In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
orders = pd.read_csv("orders.csv")

In [26]:
users = pd.read_json("users.json")

In [27]:
import sqlite3

conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# Drop table if it already exists
conn.execute("DROP TABLE IF EXISTS restaurants;")

# Re-create table
conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [29]:
orders_users = orders.merge(users, on="user_id", how="left")

final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [30]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [31]:
gold_orders = final_df[final_df["membership"] == "Gold"]
city_revenue = gold_orders.groupby("city")["total_amount"].sum()
city_revenue = city_revenue.sort_values(ascending=False)
city_revenue.head(1)

city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [32]:
cuisine_avg = final_df.groupby("cuisine")["total_amount"].mean()
cuisine_avg = cuisine_avg.sort_values(ascending=False)
cuisine_avg.head(1)

cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [33]:
user_total = final_df.groupby("user_id")["total_amount"].sum()
users_above_1000 = user_total[user_total > 1000]
count_users = users_above_1000.count()
count_users

2544

In [34]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)
revenue_by_rating = final_df.groupby("rating_range")["total_amount"].sum()
revenue_by_rating.sort_values(ascending=False).head(1)

rating_range
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

In [35]:
gold_orders = final_df[final_df["membership"] == "Gold"]
city_avg = gold_orders.groupby("city")["total_amount"].mean()
city_avg = city_avg.sort_values(ascending=False)
city_avg.head(1)

city
Chennai    808.45908
Name: total_amount, dtype: float64

In [36]:
final_df.groupby("cuisine")["restaurant_id"].nunique().sort_values()
final_df.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False)

cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [37]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])
percentage = round((gold_orders / total_orders) * 100)
percentage

50

In [42]:
df = final_df[
    final_df["restaurant_name_x"].isin([
        "Grand Cafe Punjabi",
        "Grand Restaurant South Indian",
        "Ruchi Mess Multicuisine",
        "Ruchi Foods Chinese"
    ])
]
result = df.groupby("restaurant_name_x")["total_amount"].agg(["mean", "count"])
result = result[result["count"] < 20]
result.sort_values("mean", ascending=False).head(1)

,mean,count
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [43]:
df = final_df[
    ((final_df["membership"] == "Gold") & (final_df["cuisine"] == "Indian")) |
    ((final_df["membership"] == "Gold") & (final_df["cuisine"] == "Italian")) |
    ((final_df["membership"] == "Regular") & (final_df["cuisine"] == "Indian")) |
    ((final_df["membership"] == "Regular") & (final_df["cuisine"] == "Chinese"))
]
result = df.groupby(["membership", "cuisine"])["total_amount"].sum()
result.sort_values(ascending=False).head(1)

membership  cuisine
Gold        Italian    1005779.05
Name: total_amount, dtype: float64

In [46]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.quarter
quarter_revenue = final_df.groupby("quarter")["total_amount"].sum()
quarter_revenue.sort_values(ascending=False).head(1)

quarter
3    2037385.1
Name: total_amount, dtype: float64

In [47]:
gold_orders_count = len(final_df[final_df["membership"] == "Gold"])
gold_orders_count

4987

In [48]:
hyderabad_orders = final_df[final_df["city"] == "Hyderabad"]
total_revenue = hyderabad_orders["total_amount"].sum()
round(total_revenue)

1889367

In [49]:
final_df["user_id"].nunique()

2883

In [50]:
gold_orders = final_df[final_df["membership"] == "Gold"]
avg_order_value = gold_orders["total_amount"].mean()
round(avg_order_value, 2)

797.15

In [51]:
high_rating_orders = len(final_df[final_df["rating"] >= 4.5])
high_rating_orders

3374

In [53]:
gold_df = final_df[final_df["membership"] == "Gold"]
city_revenue = gold_df.groupby("city")["total_amount"].sum()
top_city = city_revenue.idxmax()
orders_count = len(gold_df[gold_df["city"] == top_city])
orders_count

1337

In [54]:
len(final_df)


10000